## Импорт необходимых библиотек

In [1]:
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, random_split, DataLoader
from PIL import Image
import torchvision.models as models
from tqdm.notebook import tqdm
import torchvision.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torchvision.utils import make_grid

import albumentations as A
from albumentations.pytorch import ToTensor
from sklearn.model_selection import train_test_split
import os
import re
import requests

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import random
from sklearn.metrics import classification_report

## Список классов заболеваний + лейблы датасета

In [2]:
classes = ["Nutrient surplus", "Magnesium", "phosphate", "Healthy", "Phosphorous", "nitrates", "potassium", "nitrogen", "calcium", "Sulfur"]
classes_labels = {
        "Nutrient surplus": [0],
        "Magnesium+phosphate": [1, 2],
        "Healthy": [3],
        "Phosphorous+magnesium": [4, 1],
        "nitrates+potassium": [5, 6],
        "nitrogen+potassium": [7, 6],
        "calcium+phosporous": [8, 4],
        "Sulfur+magnesium": [9, 1],
    }
print(len(classes_labels), len(classes))

## Некоторые вспомогательные функции
* `seed_everything` - функция закрепляющая seed для дальнейшей воспроизводимости экспериментов
* `get_path_names` - функция для парсинга папок датасета
* `encode_label` - функция энкодинга лейблов датасета
* `decode_target` - функция декодинга ответа модели по трешхолду
* `denorm` - функция денормализации тензоров
* `show_example` - функция для отображения элемента датасета 
* `show_batch` - функция для отображения батча изображений

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
# Making a list that contains the paths of each image
def get_path_names(dir):
    images = []
    for path, subdirs, files in os.walk(data_dir):
        for sub in subdirs:
            data_path = os.path.join(path, sub)
            import pdb;pdb.set_trace()
            for p, s, f in os.walk(data_path):
                for n in f:
                    images.append(os.path.join(p, n))
    return images

def encode_label(label, classes_list = classes): #encoding the classes into a tensor of shape (10) with 0 and 1s.
    target = torch.zeros(10)
    for l in eval(label):
        target[l] = 1
    return target


def decode_target(target, threshold=0.5): #decoding the prediction tensors of 0s and 1s into text form
    result = []
    for i, x in enumerate(target):
        if (x >= threshold):
            result.append(classes[i])     
    return ' '.join(result)


def show_example(img,label):
    plt.imshow(img.permute(1, 2, 0))
    print("Label:", decode_target(label))
    print()
    print(label)
    
#let's see a batch of images (16 images) in a grid
def show_batch(dl, nmax=16):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images[:nmax], nrow=4).permute(1, 2, 0))
        break

seed_everything(42)

In [4]:
data_dir = "../input/gidtowerdatasetmultilabel/final_cropped_dataset/final_cropped_dataset"
print(os.listdir(data_dir))
print(len(os.listdir(data_dir)))

## Считывание и преобразование csv файла датасета

In [5]:
df = pd.read_csv("../input/gidtowerdatasetmultilabel/dataset.csv/dataset.csv")
df = df[["imgs", "labels"]]
df

#### В датасете есть сломанное изображение - берем все значения кроме сломанного

In [6]:
df = df[df["imgs"]!="final_cropped_dataset/Sulfur+magnesium/69.jpg"]
df = df[df["imgs"] != "final_cropped_dataset/Healthy/.DS_Store"]
df = df[df["imgs"] != "final_cropped_dataset/Nutrient surplus/.DS_Store"]
df = df[["imgs", "labels"]]
df = df.reset_index()
df

In [7]:
train_df, test_df = train_test_split(df, random_state=42, stratify=df.labels, test_size=0.15)
train_df = train_df.reset_index()
test_df = test_df.reset_index()
test_df = test_df[["imgs", "labels"]]
train_df = train_df[["imgs", "labels"]]

In [8]:
train_df

## Кастомный класс датасета

In [9]:
# A class to create a Custom Dataset that will load images and encode the labels of those images from their folder names
class myDataset(Dataset):
    def __init__(self, df, root_dir, transform=None):
        self.transform = transform
        self.df = df
        self.root_dir = root_dir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.root_dir + self.df["imgs"][idx]
        img = np.asarray(Image.open(img_path).convert("RGB"))
        labels = self.df["labels"][idx]
        if self.transform:
            img = self.transform(image=img)["image"]
        return torch.Tensor(img).permute(2, 1, 0), encode_label(labels)

## Аугментации, которые мы применим к изображениям датасета

In [10]:
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # mean and std values of the Imagenet Dataset so that pretrained models could also be used

train_transform = A.Compose([A.Resize(224, 224),
                             A.ShiftScaleRotate(0.2, 0.2, 180),
                      A.HorizontalFlip(0.5),
                      A.VerticalFlip(0.5),
                      A.HueSaturationValue(p=0.5, hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2),
#                       ToTensor(),
                      A.Normalize()])

val_transform = A.Compose([A.Resize(224, 224),
                      A.Normalize()])

In [11]:
#Creating a dataset that loads images from the specified directory, encode their labels and transforming them into tensors.
root_dir = "../input/gidtowerdatasetmultilabel/final_cropped_dataset/"

train_dataset = myDataset(train_df, root_dir=root_dir, transform = train_transform)
val_dataset = myDataset(test_df, root_dir=root_dir, transform = val_transform)
len(train_dataset)

In [12]:
show_example(*val_dataset[12]) #let's take an example

## Создание Dataloader для использования batch-ей данных при обучении

In [13]:
#setting batch size for Dataloader to load the data batch by batch
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size * 2)

### Отображение батча 

In [14]:
show_batch(val_loader)

## Класс с основными методами для Multilabel моделей

In [15]:
class MultilabelImageClassificationBase(nn.Module):
    def training_step(self, batch):
        """Get from training batch image and target, then push to model,
        finaly calculate loss
        """
        
        images, targets = batch 
        out = self(images)                            # Generate predictions
        loss = F.binary_cross_entropy(out, targets)   # Calculate loss
        return loss    

    def validation_step(self, batch):
        """Get from val batch image and target, then push to model,
        compute loss and output arrays for metric count
        """
        
        pred_Y = []
        true_Y = []
        images, targets = batch 
        out = self(images)                           # Generate predictions
        loss = F.binary_cross_entropy(out, targets)  # Calculate loss
        pred_Y.append(np.array((out.detach().to("cpu") > 0.5).float()))
        true_Y.append(np.array(targets.detach().to("cpu").tolist()))
        return {'val_loss': loss.detach(), "metrics": (np.array(pred_Y), np.array(true_Y))}      


    def validation_epoch_end(self, outputs):
        """Count scores (losses, classification metrics) at the end of val epoch"""
        
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()      # Combine losses and get the mean value
        batch_scores = [x['metrics'] for x in outputs]
        pred_Y = [preds[0] for preds in batch_scores]
        final_preds = np.concatenate(pred_Y, axis=1)[0].tolist()
        true_Y = [preds[1] for preds in batch_scores]
        final_true = np.concatenate(true_Y, axis=1)[0].tolist()
        score = classification_report(final_preds, final_true, target_names=classes)
        print(score)
        return {'val_loss': epoch_loss.item()}    

    def epoch_end(self, epoch, result):                     # display the losses
        print("Epoch [{}], last_lr: {:.4f}, train_loss: {:.4f}, val_loss: {:.4f}".format(epoch, result['lrs'][-1], result['train_loss'], result['val_loss']))

## Вспомогательные функции и класс для размещения необходимого на GPU

In [16]:
#helper functions to load the data and model onto GPU
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu') 


def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():  
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device       

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

device = get_default_device()
device

In [17]:
#loading training and validation data onto GPU
train_dl = DeviceDataLoader(train_loader, device)
val_dl = DeviceDataLoader(val_loader, device)

## Вспомогательные метода для обучения 
* `evaluate` - функция с шагом валидации модели
* `fit_one_cycle` - функция с циклом обучения модели (включая подсчет валидационных значений)

In [18]:
from tqdm.notebook import tqdm

@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []

    # Set up custom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader)) #schedule the learning rate with OneCycleLR

    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        lrs = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            # Gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()

        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

## Кастомные классы моделей 

In [19]:
from torch import nn
import torchvision as vision

class ResNet18(MultilabelImageClassificationBase):
    def __init__(self, in_channels, num_classes=10):
        super().__init__()
        self.backbone = vision.models.resnet18(pretrained=True)
        classifier_name, old_classifier = self.backbone._modules.popitem()
        if isinstance(old_classifier, nn.Sequential):
            input_shape = old_classifier[-1].in_features
            old_classifier[-1] = nn.Linear(input_shape,num_classes)

        elif isinstance(old_classifier, nn.Linear):
            input_shape = old_classifier.in_features
            old_classifier = nn.Linear(input_shape,num_classes)
        else:
            raise Exception("Uknown type of classifier {}".format(type(old_classifier)))
        self.backbone.add_module(classifier_name, old_classifier)

    def forward(self, X):
        out = self.backbone(X)
        out = F.sigmoid(out)
        return out

class MobileNetV2(MultilabelImageClassificationBase):
    def __init__(self, in_channels, num_classes=10):
        super().__init__()
        self.backbone = vision.models.mobilenet_v2(pretrained=True)
        classifier_name, old_classifier = self.backbone._modules.popitem()
        if isinstance(old_classifier, nn.Sequential):
            input_shape = old_classifier[-1].in_features
            old_classifier[-1] = nn.Linear(input_shape,num_classes)

        elif isinstance(old_classifier, nn.Linear):
            input_shape = old_classifier.in_features
            old_classifier = nn.Linear(input_shape,num_classes)
        else:
            raise Exception("Uknown type of classifier {}".format(type(old_classifier)))
        self.backbone.add_module(classifier_name, old_classifier)

    def forward(self, X):
        out = self.backbone(X)
        out = F.sigmoid(out)
        return out

In [20]:
model = to_device(MobileNetV2(3, len(classes)), device) #input size: 3, output size: 11, loading model onto GPU

## Проверка результатов коробочной (еще не обученной) модели

In [21]:
history = [evaluate(model, val_dl)]
history

### Параметры для обучения

In [22]:
epochs = 35
max_lr = 0.0001
grad_clip = 0.1
weight_decay = 1e-3
opt_func = torch.optim.Adam

In [23]:
%%time
history += fit_one_cycle(epochs, max_lr, model, train_dl, val_dl,
                         grad_clip=grad_clip,
                         weight_decay=weight_decay,
                         opt_func=opt_func)

## Сохранение модели
- В обычном .pth формате 
- В формате torchscript (конвертация модели)

In [24]:
model.to("cpu")
torch.save(model.state_dict(), "classif_model.pth")
net = torch.jit.script(model)
net.save("jit_model.pt")
model_jit = torch.load("jit_model.pt")

## Отображение результатов обучения в виде графиков

### Отображение Loss-а

In [25]:
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');
    
plot_losses(history)

### Отображение learning rate

In [26]:
def plot_lrs(history):
    lrs = np.concatenate([x.get('lrs', []) for x in history])
    plt.plot(lrs)
    plt.xlabel('Batch no.')
    plt.ylabel('Learning rate')
    plt.title('Learning Rate vs. Batch no.');
plot_lrs(history)

## Ручная проверка результатов обученной модели 

In [27]:
def predict_single(image):
    xb = image.unsqueeze(0)
    preds = model_jit(xb)
    prediction = preds[0]
    show_example(image, prediction)
predict_single(val_dataset[20][0]) #checking out the predictions of some images from the validation dataset.

In [28]:
predict_single(val_dataset[45][0])

In [29]:
predict_single(val_dataset[100][0])

In [30]:
predict_single(val_dataset[70][0])